Copyright 2018 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

## Load the relevant libraries

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

## Load the data

In [ ]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

## Data Preprocessing

In [ ]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # cast number of validation samples to an integer

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

# now we will scale the inputs
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

# We will now shuffle the data so that batching works as intended.

BUFFER_SIZE = 10000 # take samples 10000 at a time, shuffle and then take another 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples) # One batch
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))
test_inputs, test_targets = next(iter(test_data))

## Model

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')

])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [ ]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 5s - loss: 0.2743 - accuracy: 0.9211 - val_loss: 0.1515 - val_accuracy: 0.9547 - 5s/epoch - 10ms/step
Epoch 2/5
540/540 - 6s - loss: 0.1068 - accuracy: 0.9676 - val_loss: 0.0876 - val_accuracy: 0.9720 - 6s/epoch - 11ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0717 - accuracy: 0.9776 - val_loss: 0.0648 - val_accuracy: 0.9790 - 4s/epoch - 8ms/step
Epoch 4/5
540/540 - 5s - loss: 0.0549 - accuracy: 0.9823 - val_loss: 0.0517 - val_accuracy: 0.9833 - 5s/epoch - 10ms/step
Epoch 5/5
540/540 - 5s - loss: 0.0395 - accuracy: 0.9878 - val_loss: 0.0497 - val_accuracy: 0.9847 - 5s/epoch - 9ms/step


## Test the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss; {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

1/1 [==============================] - 1s 557ms/step - loss: 0.0807 - accuracy: 0.9755
Test loss; 0.08. Test accuracy: 97.55%
